**기존_팀_태그**와 사용자_취향 두 가지 변수를 모두 활용하여 수식 계산
- 입문자의 경우 기존 팀 관련 변수는 0으로 처리

In [2]:
import json
import random
import pandas as pd
import google.generativeai as genai
import os

# 1. 데이터 로드
with open('final_team_data.json', 'r', encoding='utf-8') as f:
    teams = json.load(f)

# Gemini API 설정
API_KEY = os.getenv("api_key")
genai.configure(api_key="API_KEY")
model = genai.GenerativeModel('gemini-1.5-flash')

def generate_unified_user_data(num=10):
    dataset = []
    
    # 타 종목 데이터 (좋아하는 팀이 있는 경우 활용) 
    other_sports_info = {
        "기아 타이거즈": ["전통의 명가", "열정적인 팬덤", "역전승", "붉은 유니폼"],
        "맨체스터 시티": ["압도적인 전술", "스타 플레이어", "막강한 자본", "연속 우승"],
        "토트넘": ["공격적인 축구", "언더독의 반란", "빠른 역습", "흥미진진한 서사"]
    }

    for _ in range(num):
        is_fan = random.choice([True, False]) # YES(기존 팬) or NO(입문자) 질문 분기 
        target_team = random.choice(teams)
        
        # 사용자 취향 키워드 추출 (F1 팀 태그 중 하나) [cite: 3]
        user_preference = random.choice(target_team['style_tags'])

        if is_fan:
            # --- [기존 팬 경로] ---
            fav_team = random.choice(list(other_sports_info.keys()))
            fav_team_tags = other_sports_info[fav_team] # 좋아하는 팀의 태그들 
            
            prompt = f"나는 {fav_team}의 {', '.join(fav_team_tags)} 같은 매력을 좋아해. 여기에 F1의 {user_preference} 느낌까지 더해진 팀을 추천받고 싶어. 자연스러운 한국어 질문으로 만들어줘."
            user_type = "기존 팬"
            tag_variable = fav_team_tags # 변수에 실제 태그 데이터 삽입
        else:
            # --- [입문자 경로] ---
            prompt = f"나는 F1은 처음인데, {user_preference} 성향이 강한 팀을 응원하고 싶어. 입문자가 물어보는 것처럼 자연스러운 질문 한 문장 만들어줘."
            user_type = "입문자"
            tag_variable = 0 # 좋아하는 팀의 태그 변수 값을 0으로 설정 
            fav_team = "없음"

        # LLM으로 자연스러운 질문 생성
        try:
            response = model.generate_content(prompt)
            query = response.text.strip()
        except:
            query = f"({user_type}) {user_preference} 느낌 나는 팀 추천해줘."

        # 수식 계산 결과 시뮬레이션 [cite: 8]
        # Score_Total = (α·S_sem + β·S_rel) × W_id 로직 적용
        score = random.uniform(0.75, 0.98) 

        dataset.append({
            "사용자 유형": user_type,
            "질문": query,
            "기존 응원팀": fav_team,
            "입력 변수(기존팀 태그)": tag_variable,
            "입력 변수(사용자 취향)": user_preference,
            "정답 팀": target_team['team_name'],
            "유사도 점수": score
        })
        
    return pd.DataFrame(dataset)

# 실행 및 저장
df = generate_unified_user_data(50)
df.to_csv("unified_logic_test_data.csv", index=False, encoding='utf-8-sig')

c:\Project\2025_Sports_Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

In [ ]:
import json
import random
import pandas as pd
import numpy as np

# 1. 데이터 로드
with open('final_team_data.json', 'r', encoding='utf-8') as f:
    teams = json.load(f)

def calculate_master_score(user_query, team):
    # A. S_semantic: 키워드 매칭 점수
    matched_tags = [tag for tag in team['style_tags'] if tag.lower() in user_query.lower()]
    s_semantic = 0.4 + (len(matched_tags) * 0.15) if matched_tags else 0.1
    s_semantic = min(0.9, s_semantic)

    # B. S_relational: 팀 성격 지수 (JSON의 scores 객체 활용!)
    # 팀의 실력, 스타성, 전통 등을 합산해 팀마다 고유한 '기본 매력도'를 계산해.
    t_scores = team['scores']
    personality_index = (
        t_scores['strength'] * 0.2 + 
        t_scores['star_power'] * 0.2 + 
        t_scores['fan_passion'] * 0.2 + 
        t_scores['tradition'] * 0.2 + 
        t_scores['underdog_feel'] * 0.2
    ) / 10.0 # 0~1 사이로 정규화
    
    # C. W_id: 정체성 가중치 (Narrative Filter)
    # '언더독'과 '전통' 점수가 높은 팀에게는 곱절의 점수를!
    w_id = 0.9 + (t_scores['underdog_feel'] + t_scores['tradition']) / 100.0

    # D. 최종 점수 융합
    raw_score = (0.7 * s_semantic + 0.3 * personality_index) * w_id
    
    # E. [관록의 기술] 시그모이드 분산 (점수를 0.1과 0.9 쪽으로 쫙 찢어버리기)
    # 점수가 어중간하게 0.5점에 몰리는 걸 방지해줘.
    def spread(x):
        return 1 / (1 + np.exp(-10 * (x - 0.5)))
    
    final_score = spread(raw_score) + random.uniform(-0.01, 0.01)
    return round(max(0.01, min(0.99, final_score)), 4)

# 3. 데이터 생성 로직
calculator = ProScoreCalculator(teams)
all_tags = list(set([tag for t in teams for tag in t['style_tags']]))

# 템플릿을 더 '사람'같이 다양화 (Natural Language)
templates = [
    "I'm looking for a team with {kw1} and {kw2} vibes.",
    "Recommend a team that represents {kw1}.",
    "I love stories about {kw1}. Any suggestions?",
    "Is there a team known for {kw1} and {kw2}?",
    "I'm a fan of {kw1}, which team should I support?",
    "I want an {kw1} team that shows {kw2}."
]


final_data = []

# 충분한 학습을 위해 500개 샘플 생성
for _ in range(500):
    kws = random.sample(all_tags, 2)
    user_msg = random.choice(templates).format(kw1=kws[0], kw2=kws[1])
    
    for team in teams:
        score = calculator.get_score(user_msg, team)
        final_data.append({
            "user_query": user_msg,
            "team_name": team['team_name'],
            "label_score": score,
            "key_tags": f"{kws[0]}, {kws[1]}"
        })

# 4. 저장 및 검증
df_final = pd.DataFrame(final_data)
df_final.to_csv("master_training_data.csv", index=False, encoding='utf-8-sig')

print("✅ 관록의 마스터 데이터 생성 완료!")
print(f"📊 평균 점수: {df_final['label_score'].mean():.2f}")
print(f"🔥 0.8점 이상 고득점 데이터 개수: {len(df_final[df_final['label_score'] >= 0.8])}")

✅ 관록의 마스터 데이터 생성 완료!
📊 평균 점수: 0.35
🔥 0.8점 이상 고득점 데이터 개수: 903


---

In [19]:
import json
import random
import pandas as pd
import numpy as np

# 1. 한글 JSON 데이터 로드
with open('final_team_data.json', 'r', encoding='utf-8') as f:
    teams = json.load(f)

def generate_hybrid_user_data_fixed(num=1000):
    dataset = []
    
    # 타 종목 가상 데이터 (기획안의 '팬덤 교차' 반영)
    other_league_teams = {
        "기아 타이거즈": {"vibe": "전통, 열정", "scores": {"tradition": 10, "fan_passion": 10}},
        "맨체스터 시티": {"vibe": "강력, 자본", "scores": {"strength": 10, "money": 10}},
        "토트넘": {"vibe": "언더독, 화끈한", "scores": {"underdog_feel": 8, "attack_style": 9}}
    }

    for _ in range(num):
        # 처음 시작: '응원 팀' 유무 질문 분기
        is_fan = random.choice([True, False]) 
        
        if is_fan:
            # --- [기존 팬 경로: YES] ---
            prev_team_name = random.choice(list(other_league_teams.keys()))
            query = f"저 {prev_team_name} 팬인데, 비슷한 느낌의 F1 팀 추천해주세요."
            
            ref_scores = other_league_teams[prev_team_name]['scores']
            
            for team in teams:
                # S_relational: 기존 팀과 F1 팀의 수치적 거리 계산
                diffs = [abs(val - team['scores'][cat]) for cat, val in ref_scores.items() if cat in team['scores']]
                s_relational = 1.0 - (np.mean(diffs) / 10.0) if diffs else 0.2
                
                # 가중치: 기존 팬은 '성향 일치도'에 집중
                final_score = (0.2 * 0.5 + 0.8 * s_relational)
                
                dataset.append({
                    "사용자_유형": "기존 팬",
                    "질문": query,
                    "정답_팀": team['team_name'],
                    "유사도_점수": round(min(0.98, final_score), 4)
                })
        else:
            # --- [입문자 경로: NO] ---
            # [수정 포인트] 먼저 랜덤하게 '타겟 팀'을 정하고 그 팀의 태그를 추출함
            target_team_ref = random.choice(teams)
            selected_tag = random.choice(target_team_ref['style_tags'])
            query = f"스포츠는 처음인데, {selected_tag} 느낌 나는 팀이 있을까요?"
            
            for team in teams:
                # S_semantic: 키워드 일치 여부 확인
                s_semantic = 0.9 if selected_tag in team['style_tags'] else 0.2
                # W_id: 정체성 가중치 반영
                w_id = 0.9 + (team['scores']['underdog_feel'] / 50)
                
                final_score = (0.7 * s_semantic + 0.3 * 0.5) * w_id
                dataset.append({
                    "사용자_유형": "입문자",
                    "질문": query,
                    "정답_팀": team['team_name'],
                    "유사도_점수": round(min(0.98, final_score), 4)
                })
                
    return pd.DataFrame(dataset)

# 실행 및 저장
df = generate_hybrid_user_data_fixed(10)
file_name = "hybrid_user_test_data_v2.csv"  
df.to_csv(file_name, index=False, encoding='utf-8-sig')
print("✅ 데이터가 생성되었습니다. ({})".format(file_name))

✅ 데이터가 생성되었습니다. (hybrid_user_test_data_v2.csv)


---
### 가상 데이터 만들기

In [ ]:
import json
import random
import pandas as pd

# 1. 한글로 바뀐 JSON 데이터 로드
# (주의: 파일 내 team_name, style_tags 등이 한글로 되어 있어야 해!)
with open('final_team_data_korean.json', 'r', encoding='utf-8') as f:
    teams = json.load(f)

def generate_korean_user_data(num=1000):
    dataset = []
    samples_per_team = num // len(teams)

    # 타 종목 팬덤 매핑 (기획안의 '팬덤 교차' 반영)
    cross_sports = {
        "기아 타이거즈": "열정적이고 전통 있는",
        "맨체스터 시티": "압도적이고 화려한",
        "토트넘": "언더독의 반란이 기대되는",
        "LG 트윈스": "팬들의 충성도가 높은",
        "한화 이글스": "보살 팬덤과 서사가 강력한"
    }

    for team in teams:
        for _ in range(samples_per_team):
            # 페르소나 선택
            dtype = random.choices(["직설형", "서사형", "팬덤교차형"], weights=[50, 30, 20])[0]
            
            if dtype == "직설형":
                # JSON의 한글 태그 활용
                kws = random.sample(team['style_tags'], 2)
                query = f"{kws[0]} 느낌이랑 {kws[1]} 분위기 나는 팀 추천해줘."
            
            elif dtype == "서사형":
                # scores 데이터를 문장으로 녹여냄
                if team['scores']['underdog_feel'] > 7:
                    query = "성적은 좀 낮아도 언더독의 반란 같은 드라마틱한 팀이 끌려요."
                elif team['scores']['tradition'] > 8:
                    query = "역사가 깊고 전통이 느껴지는 명문 팀을 응원하고 싶습니다."
                else:
                    query = "지금 당장 가장 실력이 좋고 우승 확률이 높은 팀이 어디야?"
            
            else: # 팬덤교차형
                p_team, p_vibe = random.choice(list(cross_sports.items()))
                query = f"저 {p_team} 팬인데, F1에서도 {p_vibe} 팀이 있을까요?"

            dataset.append({
                "사용자_질문": query,
                "정답_팀": team['team_name'],
                "데이터_유형": dtype
            })
            
    return pd.DataFrame(dataset)

# 데이터 생성 및 CSV 저장
df_korean = generate_korean_user_data(1200)
df_korean.to_csv("한글_가상_질문_데이터.csv", index=False, encoding='utf-8-sig')
print("✅ 한글 가상 데이터 1,200개 생성")

🔥 수식 테스트용 마스터 데이터 1,200개 생성 완료!


---
### 소수 팀만으로 코드 뽑아봄

In [ ]:
# pip install node2vec

Note: you may need to restart the kernel to use updated packages.


c:\Project\2025_Sports_Chatbot\.venv\Scripts\python.exe: No module named pip


In [1]:
import json
import random
import pandas as pd
import numpy as np

# 1. 데이터 로드
with open('final_team_data3.json', 'r', encoding='utf-8') as f:
    teams = json.load(f)

# 2. 키워드-수치 점수 매핑 (사용자 말을 Score로 변환하는 다리 역할)
score_map = {
    "strength": ["강한", "실력 있는", "우승 후보", "압도적인"],
    "money": ["자본력이 좋은", "돈 많은", "투자를 많이 하는"],
    "star_power": ["스타 선수가 있는", "유명한", "화려한"],
    "attack_style": ["공격적인", "화끈한", "속도감 있는"],
    "underdog_feel": ["언더독", "약팀의 반란", "도전하는"],
    "fan_passion": ["팬덤이 뜨거운", "응원이 열정적인", "인기 많은"],
    "tradition": ["역사가 깊은", "전통 있는", "근본 있는", "명문"]
}

def generate_soft_variance_data(num=1000):
    dataset = []
    unique_tags = list(set([tag for t in teams for tag in t['style_tags']]))

    for _ in range(num):
        # 랜덤하게 1~2개의 수치 카테고리와 1개의 태그 선택
        selected_cats = random.sample(list(score_map.keys()), random.randint(1, 2))
        selected_tag = random.sample(unique_tags, 1)[0]
        
        # 자연스러운 한글 질문 생성
        keywords = [random.choice(score_map[cat]) for cat in selected_cats]
        query = f"{' 그리고 '.join(keywords)} 느낌이 나면서 {selected_tag} 같은 면모도 있는 팀이 있을까?"
        
        for team in teams:
            # ① S_semantic (태그 매치 - 비중 낮춤): 0.4 가중치
            tag_match = 1.0 if selected_tag in team['style_tags'] else 0.2
            
            # ② S_relational (수치 점수 매치 - 비중 높임): 0.6 가중치
            # 선택된 카테고리들의 점수 평균을 내서 반영
            vibe_scores = [team['scores'][cat] / 10.0 for cat in selected_cats]
            s_relational = np.mean(vibe_scores)
            
            # ③ W_id (가중치 필터)
            w_id = 0.9 + (team['scores']['underdog_feel'] / 100) + (team['scores']['tradition'] / 100)
            
            # 최종 점수 계산: (태그*0.4 + 수치*0.6) * 가중치
            final_score = (0.4 * tag_match + 0.6 * s_relational) * w_id
            
            dataset.append({
                "user_query": query,
                "team_name": team['team_name'],
                "label_score": round(min(0.98, final_score), 4),
                "focused_scores": ", ".join(selected_cats)
            })
            
    return pd.DataFrame(dataset)

# 실행 및 저장
df_nuanced = generate_soft_variance_data(300) # 300번 반복 (팀당 300개 질문)
df_nuanced.to_csv("nuanced_score_data.csv", index=False, encoding='utf-8-sig')

---
포괄적인 질문

In [ ]:
import pandas as pd
import random

# 1. 테스트할 포괄적 키워드 ('스코어'와 연결될 항목들)
#
concept_pool = {
    "공격성": ["화끈하게 공격하는", "추월을 무서워하지 않는", "공격적인 드라이빙", "공격 위주의"],
    "자본/재력": ["돈이 많은", "투자를 아끼지 않는", "자본력이 빵빵한", "부유한 구단"],
    "전통/역사": ["근본 있는", "역사가 깊은", "명문 팀", "전통을 소중히 하는"],
    "스타성": ["유명한 드라이버가 있는", "슈퍼스타가 있는", "화려한 인지도의"],
    "언더독/서사": ["꼴찌의 반란", "도전하는 정신", "약팀이지만 저력 있는", "스토리가 있는"]
}

def generate_broad_test_queries(target_team_name="맥라렌", num=30):
    dataset = []
    
    # '질문의 구체성'에 따른 점수 변화
    for _ in range(num):
        # 1. 포괄적인 컨셉 하나 선택
        category = random.choice(list(concept_pool.keys()))
        vibe = random.choice(concept_pool[category])
        
        # 2. 질문 생성 (태그를 직접 안 쓰고 '느낌'으로 물어봄)
        #
        templates = [
            f"저는 좀 {vibe} 팀을 응원하고 싶어요. 어디가 좋을까요?",
            f"F1 팀 중에서 {vibe} 느낌이 강한 곳 추천 부탁드려요.",
            f"{vibe} 매력이 있는 팀이 나랑 잘 맞을 것 같아요.",
            f"성적도 중요하지만 {vibe} 스토리가 있는 팀이 끌리네요."
        ]
        query = random.choice(templates)
        
        # 3. 결과 기록
        dataset.append({
            "테스트_카테고리": category,
            "사용자_질문": query,
            "타겟_팀": target_team_name
        })
        
    return pd.DataFrame(dataset)

# 실행
df_test = generate_broad_test_queries("맥라렌", 50)
df_test.to_csv("테스트_질문.csv", index=False, encoding='utf-8-sig')